In [1]:
import pandas as pd
import psycopg2
from psycopg2 import extras

from libs.connector.postgres_connector import PostgresToDataFrameConnector
from libs.handler.postgres_handler import PostgresHandler
from libs.handler.dataframe_handler import DataFrameHandler


log_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
    
)

input_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
    schema = 'connect211_withinreach_hsds3'
)

output_handler = DataFrameHandler()

conn = PostgresToDataFrameConnector(
    input_handler=input_handler,
    output_handler=output_handler,
    log_handler=log_handler
)

print (conn)

In [2]:
get_all_tables = f"""SELECT table_name FROM information_schema.tables
       WHERE table_schema = '{conn.input_handler.schema}'"""

with conn.input_handler.connection.cursor() as cursor:
    cursor.execute(get_all_tables)
    t_names = cursor.fetchall()

print (t_names)


[('accessibility',), ('address',), ('attribute',), ('language',), ('location',), ('organization',), ('organization_identifier',), ('phone',), ('program',), ('required_document',), ('schedule',), ('service',), ('service_area',), ('service_at_location',), ('taxonomy_term',)]


In [3]:
table_map = dict()
for name in t_names:
    qry = f"SELECT column_name, data_type,is_nullable FROM information_schema.columns WHERE table_name = '{name[0]}' AND table_schema = '{conn.input_handler.schema}'"
    with conn.input_handler.connection.cursor() as cursor:
        cursor.execute(qry)
        columns = cursor.fetchall()
    table_map[name[0]] = columns
    

# Create the nodes table
if not table_map.get('nodes'):
    qry = "CREATE TABLE nodes (node_group text, attributes jsonb, aliases jsonb, legacy_keys jsonb, relations jsonb)"
    with conn.input_handler.connection.cursor(cursor_factory=extras.RealDictCursor) as cursor:
        cursor.execute(qry)
        conn.input_handler.connection.commit()

    

{'accessibility': [('tenant_id', 'character varying', 'YES'), ('id', 'character varying', 'YES'), ('description', 'text', 'YES'), ('details', 'text', 'YES'), ('location_id', 'character varying', 'YES'), ('url', 'character varying', 'YES')], 'address': [('tenant_id', 'character varying', 'YES'), ('id', 'text', 'YES'), ('location_id', 'character varying', 'YES'), ('address_1', 'character varying', 'YES'), ('address_2', 'character varying', 'YES'), ('attention', 'character varying', 'YES'), ('city', 'character varying', 'YES'), ('country', 'character varying', 'YES'), ('postal_code', 'character varying', 'YES'), ('region', 'character varying', 'YES'), ('state_province', 'character varying', 'YES'), ('address_type', 'text', 'YES')], 'attribute': [('tenant_id', 'character varying', 'YES'), ('id', 'character varying', 'YES'), ('link_id', 'character varying', 'YES'), ('taxonomy_term_id', 'character varying', 'YES'), ('link_type', 'character varying', 'YES'), ('link_entity', 'text', 'YES'), ('

In [24]:
# Check the schema for primary keys
sql = '''SELECT table_name, column_name
FROM information_schema.table_constraints
     JOIN information_schema.key_column_usage
         USING (constraint_catalog, constraint_schema, constraint_name,
                table_catalog, table_schema, table_name)
WHERE constraint_type = 'PRIMARY KEY'
ORDER BY ordinal_position;
'''
with conn.input_handler.connection.cursor() as cursor:
    cursor.execute(sql)
    primary_keys = cursor.fetchall()
    
print (primary_keys)

[('logs', 'id'), ('sanitation_logs', 'id'), ('organization_canonical', 'id'), ('addresses_canonical', 'id')]


In [16]:
# Build the dataframes
dfs = dict()
for tbl, cols in table_map.items():
    names = [col[0] for col in cols]
    name_str = ', '.join(names)
    qry = f"SELECT {name_str} FROM {tbl}"
    with conn.input_handler.connection.cursor(cursor_factory=extras.RealDictCursor) as cursor:
        cursor.execute(qry)
        my_df = pd.DataFrame.from_records(cursor.fetchall())
    
    dfs[tbl] = [my_df, cols]
    

In [18]:
print (dfs)    

{'accessibility': [                            tenant_id                                    id  \
0    b28f7cc27d004cc7a47954d1028ebc9e  ee6cb2e6-228f-540c-820b-d758966f4a90   
1    b28f7cc27d004cc7a47954d1028ebc9e  df10280e-cb58-5c51-9641-b28b91af3d12   
2    b28f7cc27d004cc7a47954d1028ebc9e  dbb23856-1e0d-54b1-a1c1-5939e2a29574   
3    b28f7cc27d004cc7a47954d1028ebc9e  4b39e485-e0e1-5ffb-bfa4-0395d0636a47   
4    b28f7cc27d004cc7a47954d1028ebc9e  c462ea02-dc59-51c1-8118-9034746bad97   
..                                ...                                   ...   
185  b28f7cc27d004cc7a47954d1028ebc9e  92d7d7d0-4396-5a7e-af41-e95c1fb1f4a8   
186  b28f7cc27d004cc7a47954d1028ebc9e  e11fcd37-c21a-5d7f-afbd-949a9b453bff   
187  b28f7cc27d004cc7a47954d1028ebc9e  8905c13d-6435-5ac1-9eae-693a4fe0ab1c   
188  b28f7cc27d004cc7a47954d1028ebc9e  84b138aa-47aa-59e1-ba1e-3a6fa067e366   
189  b28f7cc27d004cc7a47954d1028ebc9e  7d4c96f6-d05f-5856-8816-d030fa37d92b   

    description details         

In [22]:
# Iterate the dataframes and parse the data
#for df in dfs:
    # try to identify ids by column name and uniqueness
for col in dfs['address'][0].columns:
    print (f"Column {col} is {dfs['address'][0][col].is_unique}")
    if col.lower() == 'id' or col.lower() == 'address_id':
        if dfs['address'][0][col].is_unique:
            print (f'column {col} is the primary key')
        
        
        


Column tenant_id is False
Column id is True
column id is the primary key
Column location_id is False
Column address_1 is False
Column address_2 is False
Column attention is False
Column city is False
Column country is False
Column postal_code is False
Column region is False
Column state_province is False
Column address_type is False
